Андрей Панасенко https://t.me/AndreyPanasenko

# Фильмы с субтитрами по уровню английского

_**Описание**_: Просмотр фильмов на оригинальном языке - популярный и действенный метод прокачаться при изучении иностранных языков. Важно выбрать фильм, который подходит студенту по уровню сложности, т.ч. студент понимал 50 - 70 % диалогов.


_**Цель проекта**_: в течении 2х недель разработать ML решение для автоматического определения уровня сложности англоязычных фильмов.


_**Исходные данные**_
- Разметка: оценка сложности фильмов (около 200 фильмов);
- Субтитры фильмов.



## Загрузка библиотек и датасета

In [1]:
# Подключаем гугл диск

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Ссылка на датасет

https://drive.google.com/file/d/1BBgXv0iKL4LSPj8ezg0eE37PM0fvNRYb/view?usp=sharing

In [2]:
 # Загрузка датасета из гугл диска  в формате архива

 ! gdown --id 1BBgXv0iKL4LSPj8ezg0eE37PM0fvNRYb

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1BBgXv0iKL4LSPj8ezg0eE37PM0fvNRYb
To: /content/Subtitles_all.zip
100% 8.57M/8.57M [00:00<00:00, 82.6MB/s]


In [3]:
# Разархивирование датасета

! unzip /content/Subtitles_all.zip

Archive:  /content/Subtitles_all.zip
  inflating: .DS_Store               
   creating: A2/
  inflating: A2/The Walking Dead-S01E01-Days Gone Bye.English.srt  
  inflating: A2/The Walking Dead-S01E02-Guts.English.srt  
  inflating: A2/The Walking Dead-S01E03-Tell It To The Frogs.English.srt  
  inflating: A2/The Walking Dead-S01E04-Vatos.English.srt  
  inflating: A2/The Walking Dead-S01E05-Wildfire.English.srt  
  inflating: A2/The Walking Dead-S01E06-TS-19.English.srt  
   creating: B1/
  inflating: B1/AmericanBeauty1999.BRRip.srt  
  inflating: B1/Angelas.Christmas.Wish.2020.srt  
  inflating: B1/Indiana Jones And The Last Crusade DVDRip Xvid -IZON-.srt  
  inflating: B1/mechanic-resurrection_.srt  
  inflating: B1/Men.In.Black.1997.720p.Bluray.x264-SEPTiC.srt  
  inflating: B1/Rat.Race.2001.1080p.WEB-DL.DD5.1.H264-FGT.srt  
  inflating: B1/Seven.Worlds.One.Planet.S01E01.2160p.BluRay.Remux.eng.srt  
  inflating: B1/Seven.Worlds.One.Planet.S01E02.2160p.BluRay.Remux.eng.srt  
  inflat

In [4]:
# Устанавливаем библиотеки для работы с текстом

!pip install pysrt
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=d9e4d31e85d5ff73bae66010b65329ea07a5125f94dccda02799005475b98ae6
  Stored in directory: /root/.cache/pip/wheels/30/7f/e8/55de9a9b07302d9e7fe47c27910e3bea0c48536153e74bd7e6
Successfully built pysrt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=03c73a44d3865e582a56adb1879b2e4d224adc605c88f5586252e13162762c5d
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [78]:
import pandas as pd
import re
import pysrt
from pathlib import Path
import glob
import os
import sklearn
import codecs
import pymorphy2
import seaborn as sns
from pymystem3 import Mystem
from tqdm import tqdm
import matplotlib.pyplot as plt

from collections import Counter
from imblearn.under_sampling import NearMiss


from sklearn.metrics import (
    ConfusionMatrixDisplay, confusion_matrix,
    f1_score, roc_auc_score,
    classification_report, make_scorer
)
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import (
    train_test_split,
    HalvingRandomSearchCV, GridSearchCV
)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')

sns.set_style("darkgrid")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# размеченный датасет
leble_set = pd.read_excel('movies_labels.xlsx')

In [7]:
leble_set

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


In [8]:
#code for extracting file name
file_path = 'sub/10_Cloverfield_lane(2016).srt'
# the name attribute returns full name of the file
file_name = Path(file_path[:-4]).name
file_name

'10_Cloverfield_lane(2016)'

In [9]:
 sub = pysrt.open('Subtitles/10_Cloverfield_lane(2016).srt' ,encoding='iso-8859-1')

In [10]:
len(sub)

1064

Загружаем датасеты с субтитрами


In [11]:
path = r"Subtitles/" # use your path
all_files = glob.glob(os.path.join(path , "*.srt"))
print(all_files)

['Subtitles/We_are_the_Millers(2013).srt', 'Subtitles/Milada(2017).srt', 'Subtitles/Mary_Poppins_returns(2018).srt', 'Subtitles/The_break-up(2006).srt', 'Subtitles/Cars(2006).srt', 'Subtitles/Sleepless_in_Seattle(1993).srt', 'Subtitles/Dredd(2012).srt', 'Subtitles/Eurovision_song_contest_(2020).srt', 'Subtitles/Aladdin(1992).srt', 'Subtitles/Pleasantville(1998).srt', 'Subtitles/Oceans_Twelve(2004).srt', 'Subtitles/Ferdinand(2017).srt', 'Subtitles/The_usual_suspects(1995).srt', 'Subtitles/Beauty_and_the_beast(2017).srt', 'Subtitles/The_cabin_in_the_woods(2012).srt', 'Subtitles/The_greatest_showman(2017).srt', 'Subtitles/The_Ghost_Writer.srt', 'Subtitles/Breaking_Bad_The_Movie(2017).srt', 'Subtitles/The_invisible_man(2020).srt', 'Subtitles/Back_to_the_future(1985).srt', 'Subtitles/Mamma_Mia(2008).srt', 'Subtitles/Clueless(1995).srt', 'Subtitles/Moulin_Rouge(2001).srt', 'Subtitles/Hook(1991).srt', 'Subtitles/Dune(2021).srt', 'Subtitles/Powder(1995).srt', 'Subtitles/Warm_bodies(2013).srt',

In [12]:
path = r'Subtitles/' # use your path
all_files = glob.glob(os.path.join(path , "*.srt"))
li = []

for filename in all_files:
    file_name = filename[10:-4]

    sub = pysrt.open(filename,encoding='iso-8859-1')

    a =str()
    for i in range(1,len(sub)):

      a += ' ' + sub[i].text_without_tags
    df = pd.DataFrame(data = {'file_name':[file_name],'sub':[a]})
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [13]:
frame


,file_name,sub
0,We_are_the_Millers(2013),...it's full-on double rainbow all\nthe way a...
1,Milada(2017),"Thank you. So, how does it feel\nbeing back a..."
2,Mary_Poppins_returns(2018),Have come and gone Through the misty\nmorning...
3,The_break-up(2006),"The Break-Up Come on, come on, come on.\nAll ..."
4,Cars(2006),[Inhaling and exhaling deeply] [Male] OK... H...
...,...,...
110,Bridget_Joness_Baby,How in hell\ndid I end up here again? Fuck of...
111,The_blind_side(2009),LElGH ANNE: There's a moment oforderly\nsilen...
112,A_knights_tale(2001),Should we help him? He's due in the lists in ...
113,Meet_the_parents(2000),but these last ten months\nhave just been the...


In [14]:
frame.duplicated().sum()


0

In [15]:

path = r'A2/' # use your path
all_files = glob.glob(os.path.join(path , "*.srt"))
li = []
for filename in all_files:
    file_name = filename[3:-4]

    sub = pysrt.open(filename,encoding='iso-8859-1')
    a =str()
    for i in range(1,len(sub)):
      a += ' ' + sub[i].text_without_tags
    df = pd.DataFrame(data = {'file_name':[file_name],'sub':[a]})
    li.append(df)
frame_A2 = pd.concat(li, axis=0, ignore_index=True)

In [16]:
frame_A2

,file_name,sub
0,The Walking Dead-S01E04-Vatos.English,- What?\n- Nothing. It's not nothing.\nIt's a...
1,The Walking Dead-S01E02-Guts.English,Boy: Mom! Right here. ( pots rattling ) Any l...
2,The Walking Dead-S01E05-Wildfire.English,I don't know if you're out there. I don't kno...
3,The Walking Dead-S01E01-Days Gone Bye.English,( brakes squeak ) - ( engine stops )\n- ( tru...
4,The Walking Dead-S01E06-TS-19.English,"Hey hey.\nWhoa whoa whoa whoa. Ma'am. Ma'am, ..."
5,The Walking Dead-S01E03-Tell It To The Frogs.E...,"Merle:\nThat's right. You heard me, bitch. Yo..."


In [17]:

path = r'B1/' # use your path
all_files = glob.glob(os.path.join(path , "*.srt"))
li = []
for filename in all_files:
    file_name = filename[3:-4]

    sub = pysrt.open(filename,encoding='iso-8859-1')
    a =str()
    for i in range(1,len(sub)):
      a += ' ' + sub[i].text_without_tags
    df = pd.DataFrame(data = {'file_name':[file_name],'sub':[a]})
    li.append(df)
frame_B1 = pd.concat(li, axis=0, ignore_index=True)

In [18]:
frame_B1

,file_name,sub
0,Rat.Race.2001.1080p.WEB-DL.DD5.1.H264-FGT,Each and every day And I sit down at the tabl...
1,Seven.Worlds.One.Planet.S01E03.2160p.BluRay.Re...,the Andes mountains rise\nalmost vertically. ...
2,Spirit.Stallion.of.the.Cimarron.EN,"The story I want to tell you,\ncan not be fou..."
3,Indiana Jones And The Last Crusade DVDRip Xvid...,"Herman's horse-sick! Chaps, no one wander off..."
4,Valentine's.Day.2010.Subtitles.YIFY,"It's your buddy, Romeo Midnight.\nI know. I'm..."
5,Seven.Worlds.One.Planet.S01E02.2160p.BluRay.Re...,It stretches from the equator\nto beyond the ...
6,Seven.Worlds.One.Planet.S01E07.2160p.BluRay.Re...,No continent on Earth today\nhas such spectac...
7,SlingShot (2014) WEB.eng,We can empty\nhalf of all the beds in all the...
8,Terminator_2_Judgment_Day_1991_roNy,The survivors of the nuclear fire\ncalled the...
9,AmericanBeauty1999.BRRip,who's going to spray his shorts whenever\nI b...


In [19]:
frame_B1.duplicated().sum()

0

In [20]:
frame_B1

,file_name,sub
0,Rat.Race.2001.1080p.WEB-DL.DD5.1.H264-FGT,Each and every day And I sit down at the tabl...
1,Seven.Worlds.One.Planet.S01E03.2160p.BluRay.Re...,the Andes mountains rise\nalmost vertically. ...
2,Spirit.Stallion.of.the.Cimarron.EN,"The story I want to tell you,\ncan not be fou..."
3,Indiana Jones And The Last Crusade DVDRip Xvid...,"Herman's horse-sick! Chaps, no one wander off..."
4,Valentine's.Day.2010.Subtitles.YIFY,"It's your buddy, Romeo Midnight.\nI know. I'm..."
5,Seven.Worlds.One.Planet.S01E02.2160p.BluRay.Re...,It stretches from the equator\nto beyond the ...
6,Seven.Worlds.One.Planet.S01E07.2160p.BluRay.Re...,No continent on Earth today\nhas such spectac...
7,SlingShot (2014) WEB.eng,We can empty\nhalf of all the beds in all the...
8,Terminator_2_Judgment_Day_1991_roNy,The survivors of the nuclear fire\ncalled the...
9,AmericanBeauty1999.BRRip,who's going to spray his shorts whenever\nI b...


In [21]:

path = r'B2/' # use your path
all_files = glob.glob(os.path.join(path , "*.srt"))
li = []
for filename in all_files:
    file_name = filename[3:-4]

    sub = pysrt.open(filename,encoding='iso-8859-1')
    a =str()
    for i in range(1,len(sub)):
      a += ' ' + sub[i].text_without_tags
    df = pd.DataFrame(data = {'file_name':[file_name],'sub':[a]})
    li.append(df)
frame_B2 = pd.concat(li, axis=0, ignore_index=True)

In [22]:
frame_B2

,file_name,sub
0,Suits.S01E03.1080p.BluRay.AAC5.1.x265-DTG.02.EN,Offered only\nto our most exclusive members. ...
1,Suits.Episode 1- Denial,Are you doing what I think\nyou're doing? Rac...
2,Suits.Episode 14- Self Defense,What do you mean\nyou can't take the case? Je...
3,Crazy4TV.com - Suits.S06E07.720p.BluRay.x265.H...,You had an alibi witness and your public defe...
4,Suits.S02E03.HDTV.x264-ASAP,"I am armed... with a... with a big gun. Oh, m..."
...,...,...
102,"Crown, The S01E02 - Hyde Park Corner.en.SDH",-Ma'am?\n-Yes. How long do we have? We land i...
103,Virgin.River.S01E08.INTERNAL.720p.WEB.x264-STRiFE,- Any known allergies?\n- Um... When was the ...
104,Suits.S01E12.1080p.BluRay.AAC5.1.x265-DTG.02.EN,The least you can do\nis be honest with Jenny...
105,Suits.Episode 2- Compensation,I'm not quitting. I'm going to work for Louis...


In [23]:
frame_B2.duplicated().sum()

0

In [24]:

path = r'C1/' # use your path
all_files = glob.glob(os.path.join(path , "*.srt"))
li = []
for filename in all_files:
    file_name = filename[3:-4]

    sub = pysrt.open(filename,encoding='iso-8859-1')
    a =str()
    for i in range(1,len(sub)):
      a += ' ' + sub[i].text_without_tags
    df = pd.DataFrame(data = {'file_name':[file_name],'sub':[a]})
    li.append(df)
frame_C1 = pd.concat(li, axis=0, ignore_index=True)

In [25]:
frame_C1

,file_name,sub
0,Suits S04E12 EngSub,
1,Downton Abbey - S01E03 - Episode 3.eng.SDH,"They said it would,\nwhich isn't quite the sa..."
2,Downton Abbey - S01E05 - Episode 5.eng.SDH,"You made me jump. Daisy, what is the matter w..."
3,Suits S04E08 EngSub,
4,Suits S04E09 EngSub,
5,Suits S04E11 EngSub,
6,Downton Abbey - S01E02 - Episode 2.eng.SDH,For good or ill. [SIGHS] I still don't see wh...
7,Suits.S03E01.480pHDTV.x264-mSD,"You win,\nyou get what you want. You lose,\ny..."
8,Suits S04E07 EngSub,
9,Suits S04E04 EngSub,


In [26]:
frame_C1[frame_C1.duplicated(keep=False)]

,file_name,sub


In [27]:
# обьединяем датасеты с субтитрами
all_set = pd.concat([frame,frame_A2,frame_B1,frame_B2,frame_C1]).reset_index(drop=True)

In [28]:
all_set

,file_name,sub
0,We_are_the_Millers(2013),...it's full-on double rainbow all\nthe way a...
1,Milada(2017),"Thank you. So, how does it feel\nbeing back a..."
2,Mary_Poppins_returns(2018),Have come and gone Through the misty\nmorning...
3,The_break-up(2006),"The Break-Up Come on, come on, come on.\nAll ..."
4,Cars(2006),[Inhaling and exhaling deeply] [Male] OK... H...
...,...,...
273,Suits S04E16 EngSub,
274,Suits S04E10 EngSub,
275,Suits.S03E10.HDTV.x264-mSD,This is a copy of the letter you wrote a copy...
276,Suits.S03E08.480p.HDTV.x264-mSD,Darby backs me\nfor managing partner. I don't...


In [29]:
all_set.duplicated().sum()

0

In [30]:
# соединяем по названию фильма субтитры и размеченный датасет
new_df = leble_set.merge(all_set, left_on='Movie', right_on= 'file_name', how = 'left').drop(['id','file_name'],axis=1)

In [31]:
new_df[new_df['sub'].isna()]

,Movie,Level,sub
81,The Secret Life of Pets.en,B2,NaN
106,Up (2009),A2/A2+,NaN
235,Glass Onion,B2,NaN
236,Matilda(2022),C1,NaN
237,Bullet train,B1,NaN
238,Thor: love and thunder,B2,NaN
239,Lightyear,B2,NaN
240,The Grinch,B1,NaN


В данных есть пропуски в столбце субтитры, для первого взгляда на задачу рассмотрим имеющийся датасет без обогащения.Оставим заполнение пропусков на дальнейшую доработку модели.

In [32]:
new_df

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,(CLANGING) (DRAWER CLOSES) (INAUDIBLE) (CELL ...
1,10_things_I_hate_about_you(1999),B1,"I'll be right with you. So, Cameron. Here you..."
2,A_knights_tale(2001),B2,Should we help him? He's due in the lists in ...
3,A_star_is_born(2018),B2,Get to it. âª Black eyes open wide\nIt's tim...
4,Aladdin(1992),A2/A2+,Where the caravan camels roam Where it's flat...
...,...,...,...
236,Matilda(2022),C1,NaN
237,Bullet train,B1,NaN
238,Thor: love and thunder,B2,NaN
239,Lightyear,B2,NaN


In [33]:
new_df.shape

(241, 3)

## В данных есть пропущенные субтитры, в дальнейшем можно обогатить данные, добавить дополнительные фильмы.

In [34]:
# Оставляем только строки без пропусков
df_wo_nan = new_df[~new_df['sub'].isna()]

In [35]:
df_wo_nan.shape

(233, 3)

In [36]:
df_wo_nan

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,(CLANGING) (DRAWER CLOSES) (INAUDIBLE) (CELL ...
1,10_things_I_hate_about_you(1999),B1,"I'll be right with you. So, Cameron. Here you..."
2,A_knights_tale(2001),B2,Should we help him? He's due in the lists in ...
3,A_star_is_born(2018),B2,Get to it. âª Black eyes open wide\nIt's tim...
4,Aladdin(1992),A2/A2+,Where the caravan camels roam Where it's flat...
...,...,...,...
230,Suits.S03E06.720p.HDTV.x264-mSD,C1,I assume my deal\nwith Edward is dead. As lon...
231,Suits.S03E07.HDTV.x264-mSD,C1,It's going up\non the wall tomorrow... And th...
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,Darby backs me\nfor managing partner. I don't...
233,Suits.S03E09.480p.HDTV.x264-mSD,C1,I'm bonding\nwith your father here. - Speakin...


В тексте присутствуют спецсимволы, имена собственные, знаки перехода на новую строку, перейдем к их обработке

In [37]:
# разделяем по переходам на новую строку и заменяем на пробел
df_wo_nan['sub'] = df_wo_nan['sub'].apply(lambda x:' '.join(x.split('\n')))

<ipython-input-37-0bc990ccc4f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_nan['sub'] = df_wo_nan['sub'].apply(lambda x:' '.join(x.split('\n')))


In [38]:
# Выделяем и удаляем имена собственные

def extract_proper_nouns(text):
    # Токенизация слов
    words = nltk.word_tokenize(text)
    # Определение частей речи
    tagged_words = nltk.pos_tag(words)
    # Отбор имен собственных (NNP и NNPS)
    proper_nouns = [word for word, pos in tagged_words if pos not in ['NNP', 'NNPS']]

    return " ".join(proper_nouns)




In [39]:
df_wo_nan['sub'] = df_wo_nan['sub'].apply(extract_proper_nouns)

<ipython-input-39-6b547d7c3b1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_nan['sub'] = df_wo_nan['sub'].apply(extract_proper_nouns)


In [40]:
df_wo_nan

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,"( CLANGING ) ( ) ( ) ( ) : , please do n't han..."
1,10_things_I_hate_about_you(1999),B1,"I 'll be right with you . So , . Here you go ...."
2,A_knights_tale(2001),B2,Should we help him ? He 's due in the lists in...
3,A_star_is_born(2018),B2,Get to it . âª eyes open wide It 's time to t...
4,Aladdin(1992),A2/A2+,Where the caravan camels roam Where it 's flat...
...,...,...,...
230,Suits.S03E06.720p.HDTV.x264-mSD,C1,I assume my deal with is dead . As long as you...
231,Suits.S03E07.HDTV.x264-mSD,C1,It 's going up on the wall tomorrow ... And th...
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,backs me for managing partner . I do n't want ...
233,Suits.S03E09.480p.HDTV.x264-mSD,C1,I 'm bonding with your father here . - Speakin...


In [41]:
df_wo_nan['sub'][233]

"I 'm bonding with your father here . - Speaking of which -- - You 're not taking it easy on me , are you ? A little . You mean something to me . You 've always meant something to me and you always will mean something to me . What do you need , ? . I am not losing to him . You . Your blood feud or whatever it is you have with that man is what put the target on my back in the first place . I made this mess . It 's up to me to clear it up . and made a deal to take you down . And part of that deal was that I got to be the one to tell you about it . If he ca n't practice law in the country , he ca n't be a member of our firm . You set this whole thing up . set this whole thing up . And , , we may be getting a divorce , but I fully intend on keeping ownership of the house . Oh , come on , just because I have to wake up early does n't mean you should suffer . , you know what they say . The couple that wakes together ... together ? - Stays together . - Yeah , that does n't rhyme . You 're rid

In [42]:
# Приводим текст к нижнему регистру
df_wo_nan['sub'] = df_wo_nan['sub'].str.lower()

<ipython-input-42-4c0317f9a11e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_nan['sub'] = df_wo_nan['sub'].str.lower()


In [43]:
# Удаляем все символы кроме букв, апострофов(они нужны для распознавания некоторых стоп слов) и пробелов

def remove_non_alphanumeric_except_apostrophe(input_string):
    # Используем регулярное выражение для удаления всех символов, кроме букв, пробелов и апострофа
    cleaned_string = re.sub(r"[^A-Za-z ']+", "", input_string)
    return cleaned_string



In [44]:
df_wo_nan['sub'] = df_wo_nan['sub'].apply(remove_non_alphanumeric_except_apostrophe)

<ipython-input-44-52c22c34e804>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_nan['sub'] = df_wo_nan['sub'].apply(remove_non_alphanumeric_except_apostrophe)


In [45]:
df_wo_nan

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,clanging please do n't hang up just...
1,10_things_I_hate_about_you(1999),B1,i 'll be right with you so here you go sch...
2,A_knights_tale(2001),B2,should we help him he 's due in the lists in ...
3,A_star_is_born(2018),B2,get to it eyes open wide it 's time to testi...
4,Aladdin(1992),A2/A2+,where the caravan camels roam where it 's flat...
...,...,...,...
230,Suits.S03E06.720p.HDTV.x264-mSD,C1,i assume my deal with is dead as long as you ...
231,Suits.S03E07.HDTV.x264-mSD,C1,it 's going up on the wall tomorrow and this ...
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,backs me for managing partner i do n't want i...
233,Suits.S03E09.480p.HDTV.x264-mSD,C1,i 'm bonding with your father here speaking ...


In [46]:

# для лемматизация нужно обозначить тэгами части речи

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Проводим лемматизацию и токенизацию
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):

    return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]

In [47]:
df_wo_nan['sub'] = df_wo_nan['sub'].apply(lemmatize_text)

<ipython-input-47-8ed555112b9f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_nan['sub'] = df_wo_nan['sub'].apply(lemmatize_text)


In [48]:
df_wo_nan

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,"[clang, please, do, n't, hang, up, just, talk,..."
1,10_things_I_hate_about_you(1999),B1,"[i, 'll, be, right, with, you, so, here, you, ..."
2,A_knights_tale(2001),B2,"[should, we, help, him, he, 's, due, in, the, ..."
3,A_star_is_born(2018),B2,"[get, to, it, eye, open, wide, it, 's, time, t..."
4,Aladdin(1992),A2/A2+,"[where, the, caravan, camel, roam, where, it, ..."
...,...,...,...
230,Suits.S03E06.720p.HDTV.x264-mSD,C1,"[i, assume, my, deal, with, be, dead, a, long,..."
231,Suits.S03E07.HDTV.x264-mSD,C1,"[it, 's, go, up, on, the, wall, tomorrow, and,..."
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,"[back, me, for, manage, partner, i, do, n't, w..."
233,Suits.S03E09.480p.HDTV.x264-mSD,C1,"[i, 'm, bonding, with, your, father, here, spe..."


In [49]:
# Рассмотрим стоп слова
stops = set(stopwords.words('english'))
print(stops)

{'of', 'than', 'nor', 'hadn', 'each', 'here', "weren't", 's', 'his', "you'll", 'wasn', 'such', 'most', 'whom', 'you', 'ain', "don't", 'will', 'weren', 't', 'from', 'needn', 'while', 'not', 'because', 'its', 've', 'these', "isn't", 'with', 'where', 'them', 'won', 'both', "wouldn't", "wasn't", 'haven', 'only', 'my', 'it', 'having', 'same', 'down', 'me', 'being', 'then', 'once', 'what', 'before', 'd', 'after', 'few', 'himself', 'during', "haven't", 'ours', 'at', 'our', 'yourselves', 'hasn', 'against', "that'll", 'up', 'no', 'we', 'between', 'own', 'have', 'who', 'ma', 'are', 'to', 'y', 'o', 'and', 'below', 'about', "you've", 'an', "hasn't", 'very', 'theirs', 'doesn', 'the', 'which', 'been', 'for', "you'd", 'as', "shan't", 'myself', 'll', "you're", 'until', 'isn', "shouldn't", 'out', 'couldn', "hadn't", 'if', 'did', 'be', 'but', 'through', 'off', 'has', 'or', 'hers', "needn't", 'a', 'any', 'when', "she's", "it's", 'they', 'didn', 'itself', 'am', "aren't", "mightn't", 'should', 'now', 'just

In [50]:

# Удаляем стоп слова

def stop_words(text):
    stop_words = set(stopwords.words('english') + ['n']) # остались буквы n, добавим их к стоп словам
    return [word for word in text if not word in stop_words]


In [51]:
df_wo_nan['sub'] = df_wo_nan['sub'].apply(stop_words)

<ipython-input-51-b263c91e9b57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_nan['sub'] = df_wo_nan['sub'].apply(stop_words)


In [52]:
df_wo_nan

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,"[clang, please, n't, hang, talk, okay, ca, n't..."
1,10_things_I_hate_about_you(1999),B1,"['ll, right, go, school, year, brat, yeah, dad..."
2,A_knights_tale(2001),B2,"[help, 's, due, list, two, minute, two, minute..."
3,A_star_is_born(2018),B2,"[get, eye, open, wide, 's, time, testify, room..."
4,Aladdin(1992),A2/A2+,"[caravan, camel, roam, 's, flat, immense, heat..."
...,...,...,...
230,Suits.S03E06.720p.HDTV.x264-mSD,C1,"[assume, deal, dead, long, win, murder, trial,..."
231,Suits.S03E07.HDTV.x264-mSD,C1,"['s, go, wall, tomorrow, way, get, side, n't, ..."
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,"[back, manage, partner, n't, want, anymore, ge..."
233,Suits.S03E09.480p.HDTV.x264-mSD,C1,"['m, bonding, father, speak, 're, take, easy, ..."


In [53]:
# Используем регулярное выражение для удаления всех символов, кроме букв
def remove_non_alphanumeric(input_string):

    input_string = " ".join(input_string)
    cleaned_string = re.sub(r"[^A-Za-z ]+", " ", input_string)
    return cleaned_string


In [54]:
df_wo_nan['sub'] = df_wo_nan['sub'].apply(remove_non_alphanumeric)

<ipython-input-54-21da12938a3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_nan['sub'] = df_wo_nan['sub'].apply(remove_non_alphanumeric)


In [55]:
df_wo_nan

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,clang please n t hang talk okay ca n t believe...
1,10_things_I_hate_about_you(1999),B1,ll right go school year brat yeah dad s enou...
2,A_knights_tale(2001),B2,help s due list two minute two minute forfeit...
3,A_star_is_born(2018),B2,get eye open wide s time testify room lie eve...
4,Aladdin(1992),A2/A2+,caravan camel roam s flat immense heat intens...
...,...,...,...
230,Suits.S03E06.720p.HDTV.x264-mSD,C1,assume deal dead long win murder trial ll nex...
231,Suits.S03E07.HDTV.x264-mSD,C1,s go wall tomorrow way get side n t want side...
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,back manage partner n t want anymore get sight...
233,Suits.S03E09.480p.HDTV.x264-mSD,C1,m bonding father speak re take easy little m...


In [56]:
df_wo_nan['sub'] = df_wo_nan['sub'].apply(lemmatize_text)

<ipython-input-56-8ed555112b9f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_nan['sub'] = df_wo_nan['sub'].apply(lemmatize_text)


In [57]:
df_wo_nan

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,"[clang, please, n, t, hang, talk, okay, ca, n,..."
1,10_things_I_hate_about_you(1999),B1,"[ll, right, go, school, year, brat, yeah, dad,..."
2,A_knights_tale(2001),B2,"[help, s, due, list, two, minute, two, minute,..."
3,A_star_is_born(2018),B2,"[get, eye, open, wide, s, time, testify, room,..."
4,Aladdin(1992),A2/A2+,"[caravan, camel, roam, s, flat, immense, heat,..."
...,...,...,...
230,Suits.S03E06.720p.HDTV.x264-mSD,C1,"[assume, deal, dead, long, win, murder, trial,..."
231,Suits.S03E07.HDTV.x264-mSD,C1,"[s, go, wall, tomorrow, way, get, side, n, t, ..."
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,"[back, manage, partner, n, t, want, anymore, g..."
233,Suits.S03E09.480p.HDTV.x264-mSD,C1,"[m, bonding, father, speak, re, take, easy, li..."


In [58]:
df_wo_nan['sub'] = df_wo_nan['sub'].apply(stop_words)

<ipython-input-58-b263c91e9b57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_nan['sub'] = df_wo_nan['sub'].apply(stop_words)


In [59]:
df_wo_nan

,Movie,Level,sub
0,10_Cloverfield_lane(2016),B1,"[clang, please, hang, talk, okay, ca, believe,..."
1,10_things_I_hate_about_you(1999),B1,"[right, go, school, year, brat, yeah, dad, eno..."
2,A_knights_tale(2001),B2,"[help, due, list, two, minute, two, minute, fo..."
3,A_star_is_born(2018),B2,"[get, eye, open, wide, time, testify, room, li..."
4,Aladdin(1992),A2/A2+,"[caravan, camel, roam, flat, immense, heat, in..."
...,...,...,...
230,Suits.S03E06.720p.HDTV.x264-mSD,C1,"[assume, deal, dead, long, win, murder, trial,..."
231,Suits.S03E07.HDTV.x264-mSD,C1,"[go, wall, tomorrow, way, get, side, want, sid..."
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,"[back, manage, partner, want, anymore, get, si..."
233,Suits.S03E09.480p.HDTV.x264-mSD,C1,"[bonding, father, speak, take, easy, little, m..."


In [60]:
#преобразуем текст  в вектор методом  TF-IDF

tfidf_vectorizer = TfidfVectorizer()

text = df_wo_nan['sub'].apply(lambda x:  " ".join(x))
values = tfidf_vectorizer.fit_transform(text)

In [61]:
values

<233x24369 sparse matrix of type '<class 'numpy.float64'>'
	with 208283 stored elements in Compressed Sparse Row format>

In [62]:
tfidf_vectorizer.get_feature_names_out()

array(['aa', 'aaa', 'aaaaah', ..., 'zounds', 'zuk', 'zuzu'], dtype=object)

_____________________________________________
# Предобработка завершена.

- Удалены имена собственные
- Спецсимволы
- стоп слова
- Проведена лематизация и токенизация
- Данные векторизированны с помощью TF-IDF

_____________________________________________

В дальнейшем следует обратить внимание на междометия, эта обработка может увеличить качество модели

In [63]:
# Разделяем данные на таргет и признаки
X = values
y = df_wo_nan['Level'].values
# Разделяем на тенировочную и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42,stratify=y)

In [64]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((156, 24369), (77, 24369), (156,), (77,))

# Модель наивный Байес

In [65]:

nb = GaussianNB()
result_bayes = nb.fit(x_train.toarray(), y_train)
nb.score(x_test.toarray(),y_test)


0.5064935064935064

# Модель логистическая регрессия

In [66]:

logreg = LogisticRegression( )
result_logreg = logreg.fit(x_train, y_train)
logreg.score(x_test,y_test)


0.5324675324675324

# Модель метод опорных векторов

In [67]:

metodsvm = svm.SVC()
result_svm = metodsvm.fit(x_train, y_train)
metodsvm.score(x_test, y_test)


0.5714285714285714

# Модель адаптивный бустинг

In [68]:


modelClf = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=6), n_estimators=100, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.33, random_state = 42)

modelclf_fit = modelClf.fit(X_train, y_train)
modelClf.score(X_valid, y_valid)


0.6233766233766234

# Модель градиентный бустинг

In [69]:




modelClf = GradientBoostingClassifier(max_depth=7, n_estimators=600,random_state=12, learning_rate=0.1)



modelClf.fit(X_train, y_train)
modelClf.score(X_valid, y_valid)

0.6623376623376623

# Модель логистическая регрессия с поиском по сетке

In [70]:



param_grid = {
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]

              }

lr_tfidf = Pipeline([
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [71]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, *

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('clf',
                                        LogisticRegression(random_state=0))]),
             n_jobs=-1,
             param_grid={'clf__C': [1.0, 10.0, 100.0],
                         'clf__penalty': ['l1', 'l2']},
             scoring='accuracy', verbose=1)

In [72]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 100.0, 'clf__penalty': 'l2'} 
CV Accuracy: 0.527


In [73]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(x_test,y_test))

Test Accuracy: 0.844


# Посмотрим на лучшую модель в разрезе разных метрик классификации

In [74]:
Y_pred=modelClf.predict(x_test)

print(classification_report(y_test,Y_pred))
print(confusion_matrix(y_test,Y_pred))

              precision    recall  f1-score   support

          A2       1.00      1.00      1.00         2
      A2/A2+       0.75      0.75      0.75         8
  A2/A2+, B1       0.50      0.50      0.50         2
          B1       0.94      0.88      0.91        17
      B1, B2       0.43      1.00      0.60         3
          B2       0.97      0.91      0.94        32
          C1       1.00      0.92      0.96        13

    accuracy                           0.88        77
   macro avg       0.80      0.85      0.81        77
weighted avg       0.91      0.88      0.89        77

[[ 2  0  0  0  0  0  0]
 [ 0  6  0  1  1  0  0]
 [ 0  0  1  0  1  0  0]
 [ 0  1  1 15  0  0  0]
 [ 0  0  0  0  3  0  0]
 [ 0  1  0  0  2 29  0]
 [ 0  0  0  0  0  1 12]]


# Посмотрим на подозрительную модель в разрезе разных метрик классификации

In [75]:
Y_pred=gs_lr_tfidf.predict(x_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(y_test,Y_pred))
print(confusion_matrix(y_test,Y_pred))

              precision    recall  f1-score   support

          A2       1.00      0.50      0.67         2
      A2/A2+       1.00      0.75      0.86         8
  A2/A2+, B1       1.00      0.50      0.67         2
          B1       0.70      0.82      0.76        17
      B1, B2       1.00      1.00      1.00         3
          B2       0.90      0.88      0.89        32
          C1       0.80      0.92      0.86        13

    accuracy                           0.84        77
   macro avg       0.91      0.77      0.81        77
weighted avg       0.86      0.84      0.84        77

[[ 1  0  0  1  0  0  0]
 [ 0  6  0  2  0  0  0]
 [ 0  0  1  1  0  0  0]
 [ 0  0  0 14  0  2  1]
 [ 0  0  0  0  3  0  0]
 [ 0  0  0  2  0 28  2]
 [ 0  0  0  0  0  1 12]]


____________________________
# В ходе выбора модели были опробованы следующие алгоритмы с метрикой качества Accuracy:
- Логистическая регрессия (0.53)
- Наивный Байес (0.5)
- Метод опорных векторов (0.57)
- Адаптивный бустинг (0.62)
- Градиентный бустинг(0.66)
____________________________
- Модель логистической регрессии явно имеет проблемы с данными, возможно утечк (на тренировочной выборке 0.57, на тестовой 0.84)
____________________________
Лучшая модель - Градиентный бустинг
____________________________

Предсказанные классы неоднозначны. В следующей итерации необходимо провести четкие границы между уровнями английского для более точного предсказания.

Также следует обернуть алгоритм в интерфейс streamlit для удобного пользования заказчиком.


               







---


---



---


---


*В моем проекте много белых пятен и я их осознаю,  в дальнейшем, надеюсь, дополню чтобы прикрепить к портфолию.Также надеюсь что тетрадь в коллабе не проблема, тут довольно удобно работать с загрузкой данных. Спасибо за ревью!*

---

